This JN is base on Abhishek Thakur video
https://www.youtube.com/watch?v=gB1KAhL6Hvw&list=PL98nY_tJQXZln8spB5uTZdKN08mYGkOf2&index=8

In [1]:
import torch
import tqdm
print(torch.__version__)
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn import metrics

1.8.1


In [2]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        current_data = self.data[idx,:]
        current_target = self.targets[idx]
        return {
            "x": torch.tensor(current_data, dtype=torch.float),
            "y": torch.tensor(current_target, dtype=torch.float)
        }

In [3]:
class_data, class_targets = make_classification(n_samples=1000)

In [4]:
#?train_test_split

In [5]:
# run experiment on what is stratificatoin

In [6]:
train_data, test_data, train_targets, test_targets = train_test_split(class_data, class_targets, stratify=class_targets)

In [7]:
train_data.shape,test_data.shape

((750, 20), (250, 20))

In [8]:
train_dataset = CustomDataset(train_data, train_targets)
test_dataset = CustomDataset(test_data, test_targets)

In [9]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4)#, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4)#, num_workers=2)

In [10]:
len(train_dataset)

750

In [11]:
model = lambda x, w, b: x @ w + b

In [12]:
W = torch.randn(train_data.shape[1], 1, requires_grad=True)
b = torch.rand(1, requires_grad=True)

In [13]:
outputs = []
labels = []
with torch.no_grad():
    for data in test_loader:
        xtest = data['x']
        ytest = data['y']
        
        output = model(xtest, W, b)
        labels.append(ytest)
        outputs.append(output)

In [14]:
metrics.roc_auc_score(torch.cat(labels).view(-1),torch.cat(outputs).view(-1))

0.29222400000000004

In [15]:
learning_rate = 0.001
epochs = 30

In [16]:
for epoch in tqdm.notebook.tqdm(range(epochs)):
    epoch_loss = 0
    counter = 0
    for data in train_loader:
        # get data
        xtrain = data['x']
        ytrain = data['y']
        
        # print(W.grad)
        if W.grad is not None:
            W.grad_zero_()
        # calculate the output
        output = model(xtrain, W, b)
        loss = torch.mean((ytrain.view(-1) - output.view(-1))**2)
        epoch_loss = epoch_loss + loss.item()
        loss.backward()

        # print(W.grad.view(-1))
        with torch.no_grad():
            W = W - learning_rate * W.grad.data
            b = b - learning_rate * b.grad.data
        W.requires_grad_(True)
        b.requires_grad_(True)
        
        # the gradients are reset to zero by torch
        counter += 1
    print(f'Epoch {epoch} loss {epoch_loss/counter:.4f}')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0 loss 29.0474
Epoch 1 loss 11.7221
Epoch 2 loss 5.3759
Epoch 3 loss 2.6060
Epoch 4 loss 1.3126
Epoch 5 loss 0.6918
Epoch 6 loss 0.3896
Epoch 7 loss 0.2411
Epoch 8 loss 0.1677
Epoch 9 loss 0.1311
Epoch 10 loss 0.1128
Epoch 11 loss 0.1035
Epoch 12 loss 0.0988
Epoch 13 loss 0.0965
Epoch 14 loss 0.0952
Epoch 15 loss 0.0946
Epoch 16 loss 0.0943
Epoch 17 loss 0.0941
Epoch 18 loss 0.0940
Epoch 19 loss 0.0939
Epoch 20 loss 0.0939
Epoch 21 loss 0.0939
Epoch 22 loss 0.0939
Epoch 23 loss 0.0939
Epoch 24 loss 0.0939
Epoch 25 loss 0.0939
Epoch 26 loss 0.0939
Epoch 27 loss 0.0939
Epoch 28 loss 0.0939
Epoch 29 loss 0.0939


Since the data is binary we can calculate the metric **AURC**

In [17]:
outputs = []
labels = []
with torch.no_grad():
    for data in test_loader:
        xtest = data['x']
        ytest = data['y']
        
        output = model(xtest, W, b)
        labels.append(ytest)
        outputs.append(output)

In [18]:
metrics.roc_auc_score(torch.cat(labels).view(-1),torch.cat(outputs).view(-1))

0.967296

In [19]:
class LinearRegression(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(in_features=in_features, out_features=out_features, bias=True)
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [20]:
class_targets.shape

(1000,)

In [21]:
model = LinearRegression(class_data.shape[1], 1)

In [22]:
learning_rate = 0.001
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [23]:
for epoch in tqdm.notebook.tqdm(range(epochs)):
    epoch_loss = 0
    counter = 0
    for data in train_loader:
        xtrain = data['x']
        ytrain = data['y']
        # clear gradient buffer to prevent gradients carriout from prev epoches
        optimizer.zero_grad()
        
        # predict on train data
        y_pred = model(xtrain)
        loss = criterion(ytrain,y_pred)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        counter +=1
    print(f'Epoch {epoch} loss {loss/counter:.4f}')

  0%|          | 0/30 [00:00<?, ?it/s]

/home/shmalex/anaconda3/envs/pt/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/shmalex/anaconda3/envs/pt/lib/python3.7/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0 loss 0.0049
Epoch 1 loss 0.0027
Epoch 2 loss 0.0017
Epoch 3 loss 0.0012
Epoch 4 loss 0.0009
Epoch 5 loss 0.0008
Epoch 6 loss 0.0007
Epoch 7 loss 0.0006
Epoch 8 loss 0.0006
Epoch 9 loss 0.0006
Epoch 10 loss 0.0006
Epoch 11 loss 0.0006
Epoch 12 loss 0.0006
Epoch 13 loss 0.0006
Epoch 14 loss 0.0006
Epoch 15 loss 0.0006
Epoch 16 loss 0.0006
Epoch 17 loss 0.0006
Epoch 18 loss 0.0006
Epoch 19 loss 0.0006
Epoch 20 loss 0.0006
Epoch 21 loss 0.0006
Epoch 22 loss 0.0006
Epoch 23 loss 0.0006
Epoch 24 loss 0.0006
Epoch 25 loss 0.0006
Epoch 26 loss 0.0006
Epoch 27 loss 0.0006
Epoch 28 loss 0.0006
Epoch 29 loss 0.0006


In [24]:
outputs = []
labels = []
with torch.no_grad():
    for data in test_loader:
        xtest = data['x']
        ytest = data['y']
        
        output = model(xtest)
        labels.append(ytest)
        outputs.append(output)

In [24]:
metrics.roc_auc_score(torch.cat(labels).view(-1),torch.cat(outputs).view(-1))

0.967296